# Лабораторная работа №5

## Подготовка данных

Импортируем необходимые библиотеки.

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет и выбераем из него 50000 тыс. случайных записей.

In [2]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

Убераем столбцы, которые не содержат полезной информации, а также столбцы содержащие большое количество пустых значений. После этого убераем все записи, у которых есть хотя бы одно пустое значений.

In [3]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

Заменяем столбцы со строковыми значениями на соответствующие им целочисленные. Столбцы RainToday и RainTomorrow заменяем на bool-аналоги, так как они содержат всего 2 возможных значения.

In [4]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})

Отделяем таргетный столбец RainTomorrow от остального датасета.

In [5]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

Нормализуем данные в пределах от 0 до 1.

In [6]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем датасет на тествую и тренировочную части в отношении 70/30.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

*Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.*

### Дерево решений

Импортируем класс DecisionTreeClassifier для классификации датасета методом решающего дерева.

In [8]:
from sklearn.tree import DecisionTreeClassifier

Создаём объект класса DecisionTreeClassifier, реализующий метод решающего дерева.

In [9]:
des_tree_classifier = DecisionTreeClassifier()

Определяем параметры, которые мы хотим оптимизировать для DecisionTreeClassifier.
* “criterion”: этот ключ определяет критерий, используемый для измерения качества разбиения узлов в дереве.
* ‘splitter’: этот ключ определяет стратегию поиска лучшего разбиения в узле.
* ‘max_depth’: этот ключ определяет максимальную глубину дерева.

In [10]:
des_tree_params ={
    "criterion": ['gini','entropy','log_loss'],
    'splitter': ['best','random'],
    'max_depth':range(1,100),
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [11]:
des_tree_grid = GridSearchCV(des_tree_classifier, des_tree_params, cv=2, n_jobs=-1,verbose=3)
des_tree_grid.fit(X_train, y_train)

Fitting 2 folds for each of 594 candidates, totalling 1188 fits


GridSearchCV(cv=2, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 100),
                         'splitter': ['best', 'random']},
             verbose=3)

Обучаем модель на тренировочных данных.

In [12]:
best_des_tree_model = DecisionTreeClassifier(**des_tree_grid.best_params_)
best_des_tree_model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=7, splitter='random')

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [13]:
des_tree_predicted = best_des_tree_model.predict(X_test)

Вывод отчёта.

In [14]:
print('Параметры:', des_tree_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, des_tree_predicted, digits=5))

Параметры: {'criterion': 'entropy', 'max_depth': 7, 'splitter': 'random'}
Оценка:
               precision    recall  f1-score   support

       False    0.85691   0.94612   0.89931      9039
        True    0.70751   0.45203   0.55163      2606

    accuracy                        0.83555     11645
   macro avg    0.78221   0.69908   0.72547     11645
weighted avg    0.82348   0.83555   0.82150     11645



### Случайный лес

Импортируем класс RandomForestClassifier для классификации датасета методом решающего дерева.

In [15]:
from sklearn.ensemble import RandomForestClassifier

Импортируем класс RandomForestClassifier для классификации датасета методом случайного леса.

In [16]:
rand_forest_classifier = RandomForestClassifier()

Определяем параметры, которые мы хотим оптимизировать для RandomForestClassifier.
* ‘n_estimators’: этот ключ определяет количество деревьев в случайном лесу.
* ‘criterion’: этот ключ определяет критерий, используемый для измерения качества разбиения узлов в каждом дереве леса.
* ‘max_depth’: этот ключ определяет максимальную глубину каждого дерева в лесу.

In [17]:
rand_forest_params ={
    'n_estimators': np.geomspace(1,1e2,10,dtype=int),
    'criterion': ['gini','entropy','log_loss'],
    'max_depth':range(1,100)
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.

In [18]:
rand_forest_grid = GridSearchCV(rand_forest_classifier, rand_forest_params, cv=2, n_jobs=-1,verbose=3)
rand_forest_grid.fit(X_train, y_train)

Fitting 2 folds for each of 2970 candidates, totalling 5940 fits
[CV 1/2] END criterion=gini, max_depth=2, splitter=best;, score=0.823 total time=   0.3s
[CV 1/2] END criterion=gini, max_depth=21, splitter=random;, score=0.777 total time=   0.1s
[CV 2/2] END criterion=gini, max_depth=22, splitter=best;, score=0.776 total time=   0.8s
[CV 2/2] END criterion=gini, max_depth=27, splitter=random;, score=0.768 total time=   0.1s
[CV 1/2] END criterion=gini, max_depth=29, splitter=best;, score=0.778 total time=   0.3s
[CV 2/2] END criterion=gini, max_depth=31, splitter=best;, score=0.773 total time=   0.2s
[CV 2/2] END criterion=gini, max_depth=33, splitter=best;, score=0.773 total time=   0.2s
[CV 1/2] END criterion=gini, max_depth=35, splitter=best;, score=0.780 total time=   0.5s
[CV 1/2] END criterion=gini, max_depth=39, splitter=best;, score=0.777 total time=   0.5s
[CV 1/2] END criterion=gini, max_depth=42, splitter=best;, score=0.777 total time=   0.7s
[CV 1/2] END criterion=gini, max

GridSearchCV(cv=2, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': range(1, 100),
                         'n_estimators': array([  1,   1,   2,   4,   7,  12,  21,  35,  59, 100])},
             verbose=3)

Обучаем модель на тренировочных данных.

In [19]:
rand_forest_best_model = RandomForestClassifier(**rand_forest_grid.best_params_)
rand_forest_best_model.fit(X_train, y_train)

RandomForestClassifier(criterion='log_loss', max_depth=39,
                       n_estimators=np.int64(100))

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [20]:
rand_forest_predicted = rand_forest_best_model.predict(X_test)

Вывод отчёта.

In [21]:
print('Параметры:', rand_forest_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, rand_forest_predicted, digits=5))

Параметры: {'criterion': 'log_loss', 'max_depth': 39, 'n_estimators': np.int64(100)}
Оценка:
               precision    recall  f1-score   support

       False    0.86886   0.95287   0.90893      9039
        True    0.75404   0.50115   0.60212      2606

    accuracy                        0.85178     11645
   macro avg    0.81145   0.72701   0.75552     11645
weighted avg    0.84316   0.85178   0.84027     11645

